In [94]:
from common import *

In [95]:
db = load_db(download_tar("http://vision.lems.brown.edu/sites/default/files/216db.tar.gz"))

216db exists. Skipping


In [173]:
def test_queries(descriptor, distance, top_n=5):

    desc = [descriptor(im) for im in db.image]
    db["desc"]=desc

    queries = []
    results=[]
    for clas in db.clas.unique():
        row = db.query(f"clas=='{clas}'").sample(1).iloc[0]

        C = np.stack(db.desc.to_numpy())

        desc = row.desc
        dists = distance( row.desc, C)

        queries.append(pd.DataFrame(data=[row])[["clas","image"]])
        results.append(db.iloc[np.argsort(dists)].head(n=top_n)[["clas","image"]])
    
    return queries, results

def query_acc(query, result):
    return np.sum(result.clas == query.clas.iloc[0])/len(result)

def query_accuracies(queries, results):
    res = pd.DataFrame()
    for q, r in zip(queries,results):
        res[f"{q.clas.iloc[0]}"]=pd.Series(query_acc(q,r))
    res.index.rename("Accuracy", inplace=True)
    return res

In [174]:
def show_query_results(queries, results):
    qDf = {}
    rDf = pd.DataFrame()
    for i, (q, r) in enumerate(zip(queries, results)):
        qDf[f"{q.clas.iloc[0]}"] = q.image.iloc[0]
        rDf[f"{q.clas.iloc[0]}"] = r.image.reset_index(drop=True)

    print("queries:")
    displayImages(pd.DataFrame(data=[qDf]))
    print("results:")
    displayImages(rDf)

def show_query_accuracy(queries, results):
    print("Accuracies:")
    acc = query_accuracies(queries,results)
    display(acc)
    print(f"Average along classes: {acc.transpose().mean().iloc[0]:.2f}")

In [175]:
def hu_moments(im):
    im = 255*(im==0).astype(np.uint8)
    h=cv2.moments(im)

    return cv2.HuMoments(h).flatten()

def hu_moments_log(im):
    hu = hu_moments(im)
    return -1* np.copysign(1.0, hu) * np.log10(abs(hu))


def l1Dist(vec, dataset):
    return np.linalg.norm( vec.reshape(1,-1) - dataset , axis=-1, ord=1)

In [181]:
queries, results = test_queries(hu_moments, l1Dist, top_n=5)
show_query_results(queries, results)
show_query_accuracy(queries, results)

queries:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABaCAAAAACuizbEAAADAklEQVRoBb3BAWKqWADAwOT+h84CCjx/RVmlnTH+nPHnjD9nXMHMOMf4ivwj3jK+IT/FG8bH5EC8ZHxKjsULxmfktThknGE8kPfigPGW3MSdnBLPGW/I5+IZ4zX5RjxhvCZfiZ+MV+Rb8YPxglwg/mG8IBeIfxgvyBXikXFMrhEPjGNykRgZh+QqMTIOyWViYByR68TAOCIXip1xRC4UO+OIXCk2xgG5VGyMA3KtWBkH5FqxMsCEeCBXizsDmcUDuVrcGSCTGMn14sYAmcVArhc3xkQmsZPfEAtjJpPYyG+IhbEQiJX8jpgZC4FYya+IhbGQSdzJ74iZsRCIO/kFCTEzFgJxJ78ghJgYM5nEnfyCkJgZM4FYyU1ynZCYGTOBWMki5CYJ5BshMTMmMomVLEJuYiFfCCEmxkQgVrIImcVKvhBCTIyJTOJOFiGzWMkXQoiJMRGIlSziX/KFQGJigMziTmbxg3whJGbGRCDuZBY/yRcCiYkxkVksZBY/yecCJCbGRGYxkUU8IZ8LkJgYE/lXPCGfi4kEGDN5FE/IF2IiAcZCRvGMfCFmEhh3soln5AuxM1ZyF0/JF2JnDMw4Il+InXGWfCF2xlnyUiBHYmCcJKOQR4EciYFxluwC5EEgR2JgnCWDQB4EyCYZxMA4SQYxkUFMZBMyiJ1xluxiIoOYyCqQQeyMk2QXC9nFRFYBsoudcZLsYiG7mMgqJrKLjXGS7GIhu5jIXcxkEzvjJNnEnWxiIndxI6vYGCfJJm5kFyCrWMgmNsZJsok72QTIJmayi5VxkqziTnYBsoob2cTKOElWsZJNIJu4kU2sjHNkFRvZBLKJG9nEyjhFNrGRTSCruJFdrIwzZBcb2YTsYiaDWBnvySB2sgnZxEIGsTLeklEMZBWyioWMYmW8Iw9iIKtAFnEjo9gY78iDGMgqfpBBbIx3ZBQD2cRPsoqB8Z6sYiCDeEaSeGScIxAPZBCnGZ+SUZxmfEpGcZrxMdnFecbn5C7+D+MrZvxPxp8z/pzx5/4DDuAwEO6EIqIAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAH4AAABJCAAAAADeN81BAAAB0UlEQVRoBcXBAWKiQADAwOT/j85pRVx60opQnDE+yfgk45OMTzIOYPKQ8SJjH5OnMn5lvE9+Ez8z3iSviR8Y75ANYpXxDtki1hibyWbxnLGVbBfPGZvIm+IZYwt5WzxhbCB7xH+M18ku8R/jZbJTfGe8SnaLb4xXyW7xjfESOUQsGa+Qo8TIeIEcJkbGb+RQMTC+McyMGzlWDIyRjALkaPFgjGQh5HDxYNzJSWJmfDE5STwYF3KmmBkg54o7A+RccWeAnCwmBsjJYmKAnCwmBsjZ4sYAOVvcGBdysrgxruRccWN8kVPFjXEjZ4obYyZnigvjQU4UF8aDnCnAWJDzBMZ3coKMC+MJ+TsxMFbIn4iRsU6OFyNjlfyFGBir5A/EyFglfyEGxioZJG8KWYiZsUpm8UU2iwtZiDtjlcxiIlvERBZiYqySWQxkRchF/EcWYmKsk7t4yjCM38lCfDFWySz2kqW4MtbJXewmS3FhrJO72E9GcWWsk7vYT0ZxZfxAJnEEGcSF8QOZxAFkFBfGKpnFAWQQV8YqmcV+MoorY4UMYjcZxI2xQgaxmwzixnhORrGf3MWd8ZwM4gAyiZmxRiZxELmKB+NExpLxScYnGZ/0D5gQsQFVm4eHAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA2CAAAAADRvfnlAAABR0lEQVRYCb3BAWKiQADAwOT/j86pCNWKXkHYGWM4YzhjOONrZmZmZmZ8YOxi8hfxythINosnxgayV/ww/ky+EAvjj+QrsTC2kN1iZmwhu8XM2ET2ipmxiewVM2ML2S1mxiayV8yMNRKvJNkrZsYKk5iZmck3YmasEpIjxcxYIYeLhfFKThAz45WcIGbGE4HkBDEzHsiJ4s6YybnizriTs8XEmMjpkrgwbmSIuDBuZIS4MiYyRIAxkSECjImcLibGjZwvIcC4MBkiLgyQcQIDGSfAABkkCQyQAcK4Ma7kdDEzbuRYgTyKhTGR/4gr+SDjhzyKhXEnn8SdPMnMWCEP4odxJ5/EdrKIB8ZC3osd5C4eGQ/kKvktdhIynhgr5Jc4kLFCnsWRjBXyLI5krJEncSRjnSziWMYbMomjGW+ZGYczhjOGM4b7B4HejwHRNdIJAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA5CAAAAADCN1BJAAABPklEQVRYCb3BCXLbQBDAQOD/j0aWVOLyKdHSZLuN/Yz9jP2M/Yz9jDtkSQhjjPGZYcYi78QU4yP5UQwx3pF7YohxI8l9McS4kcdihnEjF8QI4yRXxAjjJFfECOMgF8UA4yTXxADjINfEBOMgF8UAY5HL4nXGIr8RLzIWeVKSfC9+YizyXySZZLxnHGSHuDEW2SNOxiKbxMFYZJsAY5FtAoxFtgkwFtkmwFhkl1iMRXaJxVhkjzgZi+wRJ2OR14U8EidjkRcFyCNxYyzyvHgjX4QsCcRfxiLPyPieZIBxMt4zFvmF5BBPMxa5JkYYB7kiZhiLXBIzjJM8FFOMg1wQQ4yDPBRjjIM8FlOMkzwSY4wbeSDGGP/IPTHHeCN3xBzjA/lBzDE+ke/EIOMr+ShmGV9J8iamGfsZ+xn7Gfv9AW8vgQER3yerAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAIIAAACCCAAAAACvaE8hAAAC80lEQVR4AcXBAY7bOADAQPL/j+a1V2DjbGRHcmxkxvg249uMbzO+zRiTf+JuxpA8xK2MAXkSdzJeyS9xI+OFvIj7GL/JQNzGeCZDcRvjieyIuxgbsivuYmzIvriJ8UOOxE2MH3IkbmI8SJjJSNzDGJCBuIcxJK/iFsaQvIpbGGPyIm5h7JDf4hbGDnkRdzB2yIu4g7FDXsQdjB3yIu5g7JAXcQdjh7yIOxg75FXcwBiTgbiBMSYDcQNjTEbiesaYDMQNjDEZiesZQzIU1zPGZCguZ4zJUFzOGJIdcTVjSHbE1Ywh2RFXM4ZkT1zMGJFdcTFjRPbFtYwR2RfXMkbkQFzKGJEDcSljRI7ElYwBORRXMkbkUFzIGJBjcSFjRI7FdYwBeScuYwzIW3EVY0TeiosYAzIhrmEMyIy4hDEgM+ISxoBMiSsYIzIlLmAMyKT4nDEgs+JjxojMik8ZAzItPmWMyLT4kDEgC+IzxogsiI8YI7IiPmGMyJL4gDEkK+IDxpAsifOMIVkS5xljsiROM8ZkTZxljMmiOMnYIYviHGOHrIpTjD2yKE4xdsmqOMHYJctinbFPlsUy44Asi1XGEVkWi4wjsi7WGIdkXSwx3pBlscJ4Q06IecZbsizmGe/JuphlTJB1McmYIetijjFF1sUUY46sixnGJDkh3jMmyRnxljFLzoh3jFlyThwzpsk5cciYJyfFAWOBnJOxy1ggZ8UuY4WcFjuMFfKBGDKWyFYg82LEWCJP4g+ZFq+MNbIV/5jMy

results:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABaCAAAAACuizbEAAADAklEQVRoBb3BAWKqWADAwOT+h84CCjx/RVmlnTH+nPHnjD9nXMHMOMf4ivwj3jK+IT/FG8bH5EC8ZHxKjsULxmfktThknGE8kPfigPGW3MSdnBLPGW/I5+IZ4zX5RjxhvCZfiZ+MV+Rb8YPxglwg/mG8IBeIfxgvyBXikXFMrhEPjGNykRgZh+QqMTIOyWViYByR68TAOCIXip1xRC4UO+OIXCk2xgG5VGyMA3KtWBkH5FqxMsCEeCBXizsDmcUDuVrcGSCTGMn14sYAmcVArhc3xkQmsZPfEAtjJpPYyG+IhbEQiJX8jpgZC4FYya+IhbGQSdzJ74iZsRCIO/kFCTEzFgJxJ78ghJgYM5nEnfyCkJgZM4FYyU1ynZCYGTOBWMki5CYJ5BshMTMmMomVLEJuYiFfCCEmxkQgVrIImcVKvhBCTIyJTOJOFiGzWMkXQoiJMRGIlSziX/KFQGJigMziTmbxg3whJGbGRCDuZBY/yRcCiYkxkVksZBY/yecCJCbGRGYxkUU8IZ8LkJgYE/lXPCGfi4kEGDN5FE/IF2IiAcZCRvGMfCFmEhh3soln5AuxM1ZyF0/JF2JnDMw4Il+InXGWfCF2xlnyUiBHYmCcJKOQR4EciYFxluwC5EEgR2JgnCWDQB4EyCYZxMA4SQYxkUFMZBMyiJ1xluxiIoOYyCqQQeyMk2QXC9nFRFYBsoudcZLsYiG7mMgqJrKLjXGS7GIhu5jIXcxkEzvjJNnEnWxiIndxI6vYGCfJJm5kFyCrWMgmNsZJsok72QTIJmayi5VxkqziTnYBsoob2cTKOElWsZJNIJu4kU2sjHNkFRvZBLKJG9nEyjhFNrGRTSCruJFdrIwzZBcb2YTsYiaDWBnvySB2sgnZxEIGsTLeklEMZBWyioWMYmW8Iw9iIKtAFnEjo9gY78iDGMgqfpBBbIx3ZBQD2cRPsoqB8Z6sYiCDeEaSeGScIxAPZBCnGZ+SUZxmfEpGcZrxMdnFecbn5C7+D+MrZvxPxp8z/pzx5/4DDuAwEO6EIqIAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAH4AAABJCAAAAADeN81BAAAB0UlEQVRoBcXBAWKiQADAwOT/j85pRVx60opQnDE+yfgk45OMTzIOYPKQ8SJjH5OnMn5lvE9+Ez8z3iSviR8Y75ANYpXxDtki1hibyWbxnLGVbBfPGZvIm+IZYwt5WzxhbCB7xH+M18ku8R/jZbJTfGe8SnaLb4xXyW7xjfESOUQsGa+Qo8TIeIEcJkbGb+RQMTC+McyMGzlWDIyRjALkaPFgjGQh5HDxYNzJSWJmfDE5STwYF3KmmBkg54o7A+RccWeAnCwmBsjJYmKAnCwmBsjZ4sYAOVvcGBdysrgxruRccWN8kVPFjXEjZ4obYyZnigvjQU4UF8aDnCnAWJDzBMZ3coKMC+MJ+TsxMFbIn4iRsU6OFyNjlfyFGBir5A/EyFglfyEGxioZJG8KWYiZsUpm8UU2iwtZiDtjlcxiIlvERBZiYqySWQxkRchF/EcWYmKsk7t4yjCM38lCfDFWySz2kqW4MtbJXewmS3FhrJO72E9GcWWsk7vYT0ZxZfxAJnEEGcSF8QOZxAFkFBfGKpnFAWQQV8YqmcV+MoorY4UMYjcZxI2xQgaxmwzixnhORrGf3MWd8ZwM4gAyiZmxRiZxELmKB+NExpLxScYnGZ/0D5gQsQFVm4eHAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA2CAAAAADRvfnlAAABR0lEQVRYCb3BAWKiQADAwOT/j86pCNWKXkHYGWM4YzhjOONrZmZmZmZ8YOxi8hfxythINosnxgayV/ww/ky+EAvjj+QrsTC2kN1iZmwhu8XM2ET2ipmxiewVM2ML2S1mxiayV8yMNRKvJNkrZsYKk5iZmck3YmasEpIjxcxYIYeLhfFKThAz45WcIGbGE4HkBDEzHsiJ4s6YybnizriTs8XEmMjpkrgwbmSIuDBuZIS4MiYyRIAxkSECjImcLibGjZwvIcC4MBkiLgyQcQIDGSfAABkkCQyQAcK4Ma7kdDEzbuRYgTyKhTGR/4gr+SDjhzyKhXEnn8SdPMnMWCEP4odxJ5/EdrKIB8ZC3osd5C4eGQ/kKvktdhIynhgr5Jc4kLFCnsWRjBXyLI5krJEncSRjnSziWMYbMomjGW+ZGYczhjOGM4b7B4HejwHRNdIJAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA5CAAAAADCN1BJAAABPklEQVRYCb3BCXLbQBDAQOD/j0aWVOLyKdHSZLuN/Yz9jP2M/Yz9jDtkSQhjjPGZYcYi78QU4yP5UQwx3pF7YohxI8l9McS4kcdihnEjF8QI4yRXxAjjJFfECOMgF8UA4yTXxADjINfEBOMgF8UAY5HL4nXGIr8RLzIWeVKSfC9+YizyXySZZLxnHGSHuDEW2SNOxiKbxMFYZJsAY5FtAoxFtgkwFtkmwFhkl1iMRXaJxVhkjzgZi+wRJ2OR14U8EidjkRcFyCNxYyzyvHgjX4QsCcRfxiLPyPieZIBxMt4zFvmF5BBPMxa5JkYYB7kiZhiLXBIzjJM8FFOMg1wQQ4yDPBRjjIM8FlOMkzwSY4wbeSDGGP/IPTHHeCN3xBzjA/lBzDE+ke/EIOMr+ShmGV9J8iamGfsZ+xn7Gfv9AW8vgQER3yerAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAIIAAACCCAAAAACvaE8hAAAC80lEQVR4AcXBAY7bOADAQPL/j+a1V2DjbGRHcmxkxvg249uMbzO+zRiTf+JuxpA8xK2MAXkSdzJeyS9xI+OFvIj7GL/JQNzGeCZDcRvjieyIuxgbsivuYmzIvriJ8UOOxE2MH3IkbmI8SJjJSNzDGJCBuIcxJK/iFsaQvIpbGGPyIm5h7JDf4hbGDnkRdzB2yIu4g7FDXsQdjB3yIu5g7JAXcQdjh7yIOxg75FXcwBiTgbiBMSYDcQNjTEbiesaYDMQNjDEZiesZQzIU1zPGZCguZ4zJUFzOGJIdcTVjSHbE1Ywh2RFXM4ZkT1zMGJFdcTFjRPbFtYwR2RfXMkbkQFzKGJEDcSljRI7ElYwBORRXMkbkUFzIGJBjcSFjRI7FdYwBeScuYwzIW3EVY0TeiosYAzIhrmEMyIy4hDEgM+ISxoBMiSsYIzIlLmAMyKT4nDEgs+JjxojMik8ZAzItPmWMyLT4kDEgC+IzxogsiI8YI7IiPmGMyJL4gDEkK+IDxpAsifOMIVkS5xljsiROM8ZkTZxljMmiOMnYIYviHGOHrIpTjD2yKE4xdsmqOMHYJctinbFPlsUy44Asi1XGEVkWi4wjsi7WGIdkXSwx3pBlscJ4Q06IecZbsizmGe/JuphlTJB1McmYIetijjFF1sUUY46sixnGJDkh3jMmyRnxljFLzoh3jFlyThwzpsk5cciYJyfFAWOBnJOxy1ggZ8UuY4WcFjuMFfKBGDKWyFYg82LEWCJP4g+ZFq+MNbIV/5jMy

Accuracies:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
Accuracy,,,,,,,,,,,,,,,,,,
0,0.6,0.8,0.4,0.4,0.2,0.2,0.8,0.4,1.0,0.6,0.4,0.6,0.8,0.4,0.4,0.2,0.8,0.4


Average along classes: 0.52


In [190]:
queries, results = test_queries(hu_moments_log, l1Dist, top_n=5)
show_query_results(queries, results)
show_query_accuracy(queries, results)

queries:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABVCAAAAABf3YQRAAAC9klEQVRoBb3BiWHiQADAQKn/onVr42chkBAgN2P8d8Z/Z/x3xi8ZbzJ+QxbxFuMJZnKIifFLxo/kjgSSXRLPMX4gvxBPML4nvxQ/Mb4lL4hvGd+RV8VDxjfkLXGX8Q15T9xjPCbviy+Mx+QD4pbxkHxE3DAeks+Ia8Yj8ilxxXhEPiWuGI/Ix8TMeEQ+JmbGA/JBMTEekE+Kk/GAfFKcjPvks+JggBk35LPiYIAMcUU+LHYGsoqJfFrsDJBFnOTjYmeALOIknxcbYyEQJ/m82BgrgdjJH4iNsRKIjfyJuDBWArGRPxEXxkogNvIn4sJYyBAb+RNxYSxkiAv5G3FhLARiI38kVsYgQ2zkjkDeFCtjEIidfBGDvClWxiBDbORaXMibYmUMArGTK7GRNyTEygBZxEZ2CbGTNyTEyhgEYiOHJE7yshBiZQwyxIVsAomDvC4QYmEMMsRKdoFxktcFQiyMQS6SU9yQ1wUSK2MhX8QteVmAEAtjIV/EDXldgBALYyU34pa8LkCIhbGRWXwhL4uFEINxMrmIL+QNMQgxGDNZxR3yslgIMRgzQyC+ku+FPBIricG4IXGPTJJrAfJAXAgBxjWJu2QSyJVAHoiNBBhPkkkgswA5JLO4kADjSXKKQSYxyC5kEhdCYDxLDjHIJEB2AXKKmfEkOcUgkxhkFyCHmBlPklMs5BSDbGKQU0yM58gpVnKKQTaxkENMjOfIKVZyikE2sZBDTIznyCEu5BSDbGIlu5gYz5FDXMgpBtnESg5xMp4ju9jJIQbZxEoOcTKeI7vYySEG2cRKTnEwniO72MkuBjnESnZxMp4iuzjILhayiZWc4mA8Qw5xkF0MsouFnOJk/EwmcZBdDHIIkEmcjB/JJCayiUF2McgkTsZPZBYT2QXIJgaZxcn4gVyJiezilkxiYvxEJjGRQ3whu7hi/EgOcZJJ3GECccN4ikBckUk8z3iRzOJ5xqvkFL9gvEpO8QvGy2QTv2K8w4zfMv47478z/rt/WxorEMqsV+UAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAH4AAABJCAAAAADeN81BAAAB0UlEQVRoBcXBAWKiQADAwOT/j85pRVx60opQnDE+yfgk45OMTzIOYPKQ8SJjH5OnMn5lvE9+Ez8z3iSviR8Y75ANYpXxDtki1hibyWbxnLGVbBfPGZvIm+IZYwt5WzxhbCB7xH+M18ku8R/jZbJTfGe8SnaLb4xXyW7xjfESOUQsGa+Qo8TIeIEcJkbGb+RQMTC+McyMGzlWDIyRjALkaPFgjGQh5HDxYNzJSWJmfDE5STwYF3KmmBkg54o7A+RccWeAnCwmBsjJYmKAnCwmBsjZ4sYAOVvcGBdysrgxruRccWN8kVPFjXEjZ4obYyZnigvjQU4UF8aDnCnAWJDzBMZ3coKMC+MJ+TsxMFbIn4iRsU6OFyNjlfyFGBir5A/EyFglfyEGxioZJG8KWYiZsUpm8UU2iwtZiDtjlcxiIlvERBZiYqySWQxkRchF/EcWYmKsk7t4yjCM38lCfDFWySz2kqW4MtbJXewmS3FhrJO72E9GcWWsk7vYT0ZxZfxAJnEEGcSF8QOZxAFkFBfGKpnFAWQQV8YqmcV+MoorY4UMYjcZxI2xQgaxmwzixnhORrGf3MWd8ZwM4gAyiZmxRiZxELmKB+NExpLxScYnGZ/0D5gQsQFVm4eHAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA0CAAAAACcdVjuAAABT0lEQVRYCb3BB2KjAADAMPv/j/a1EFYz4QKScTnjcsbljC8yMzMzMx4w/o/JW7FmHGFIskNMjH0kOSBmxufkP8TE+JwcFzNjDzkqZsYOcljMjD3kqJgZO8hhMTM+J4fFwnhE4p4cFzPjESH+MDkuZsYjQpIkSfIjOSwWxgPybbEw7snXxcK4JyeJX8aWnCcGxoacKAbGhpwoBsbC5EwxMGZyshgYN3K6GBgjOV8MjJGcLwbGSM4XA2MiZ4uBcSOni4ExkksEGAgh1wgMkMsEGCCXCTBArhBCgIFcISbGQE4XN8ZIjkrei4Uxko/EA/JOrBgjeSfjMVkL5I9YMW7ktXhOVgLZijXjRl6L52QRIFuxZsxkkWzFSzKKgazFhjGTSSBb8YZATGQRW8aa/IofshJ7yST+MJ6RRewnv+KO8ZRM4puMF8yMLzMuZ1zOuNw/NtGOAeszs6kAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA4CAAAAAAJa4PsAAABQElEQVRYCb3BAXLaMAAAwbv/P/pqYeOhCQkuqNo11jPWM9Yz1jPWM56SU8xmfGPyKCYzvpInYiLjb/JczGM8MPlJTGOcTH4Rsxh38ruYxTjJCzGHcZDXYgrjIK/FFMZOLokJjJ1cE58zdnJNfM7YyTXxOWOQy+Jjxo1cljyKf2bcyBxxMEAgvjEG+e/iZGxkidgZg6wRg7GRNeLGGGSR2BiDrBJgDLJKgDHIIrExNrJKbIxBFomNsZE14sbYyBKxMwaZIG7kB3EwBvlcHGQXIKe4Mwb5SELcCQkxyCFOxiBvibcYO/l38R5jJ9fEIJt4k7GTK+IgxLuMnVwQJ+NtxkFeijmMO3khJjHu5IWYxLiTX8U0xsnkRzGP8UCSZzLmMb6QJ2Im4yv5KuYyvjF5FJMZT8ghIWYz1jPWM9Yz1vsD0MZ8Aeb1ugwAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABsCAAAAAB8qNJ0AAACX0lEQVRoBb3BAYLhQAAAwe7/P7pvHWGRxJCxVcafM/6c8eeMO2YY32TcyE18jXElv8W3GAu5F19iXMij+A7jTJ7FVxhnsiK+wTiTNfEFxomsiy8wQDbFfAbIppjPANkU8xnItpjPANkW0xn/yYaYzjiTdTGdcSbrYjrjQtbEfMaFrIrpjIWsiemMhayK2YyFrIvJjCvZEFMZV7IpJjJuZF9MYVzJa3GccSUj4ijjRsbEIcaVDIsDjCt5S3zIuJJ3xSeMhXwg3mcs5CPxLmMhH4r3GAv5VLzFuJAD4g3GQg6IccaFHBLDjIUcE4OMCzkoBhkLOSqGGBdyXIwwFjJBvGYsZIZ4yVjIFPGKsZA54gXjSuaIfcaVzBH7jCuZJHYZVzJL7DFuZJLYY/wik8QO4xeZJbYZv8ksscm4I7PEFuOeTBJbjHsyTawzHsg0scp4JLPEKuOJzBJrjCcyTawwnsk08cxYIdPEE2OVzBKPjFUyT9wz1sk8ccfYItPEb8YmmSdujG0yT1wZe2SaWBi7ZJq4MPbJNHFmvCKzxH/GazJJnBgjZIo4McbIuEzWBRijZEBcyKoAY5TsizuyJsAYJdvimawJjGFyEoNkRYAxTE5ilDwLMIbJjxgnzwJjlJzEOHkSYIySk3iDPAowRsmPeIs8CDAGyUm8Rx4ExiD5L94mNwHGIPkRn5Gz+GEMEojPCfGfMUYgpjDGCMQUxhD5EVMY+4TkImYw9sidmMHYJffiOGOLrIjjjA2yLo4yNsizMA4z1

results:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
0,"<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABVCAAAAABf3YQRAAAC9klEQVRoBb3BiWHiQADAQKn/onVr42chkBAgN2P8d8Z/Z/x3xi8ZbzJ+QxbxFuMJZnKIifFLxo/kjgSSXRLPMX4gvxBPML4nvxQ/Mb4lL4hvGd+RV8VDxjfkLXGX8Q15T9xjPCbviy+Mx+QD4pbxkHxE3DAeks+Ia8Yj8ilxxXhEPiWuGI/Ix8TMeEQ+JmbGA/JBMTEekE+Kk/GAfFKcjPvks+JggBk35LPiYIAMcUU+LHYGsoqJfFrsDJBFnOTjYmeALOIknxcbYyEQJ/m82BgrgdjJH4iNsRKIjfyJuDBWArGRPxEXxkogNvIn4sJYyBAb+RNxYSxkiAv5G3FhLARiI38kVsYgQ2zkjkDeFCtjEIidfBGDvClWxiBDbORaXMibYmUMArGTK7GRNyTEygBZxEZ2CbGTNyTEyhgEYiOHJE7yshBiZQwyxIVsAomDvC4QYmEMMsRKdoFxktcFQiyMQS6SU9yQ1wUSK2MhX8QteVmAEAtjIV/EDXldgBALYyU34pa8LkCIhbGRWXwhL4uFEINxMrmIL+QNMQgxGDNZxR3yslgIMRgzQyC+ku+FPBIricG4IXGPTJJrAfJAXAgBxjWJu2QSyJVAHoiNBBhPkkkgswA5JLO4kADjSXKKQSYxyC5kEhdCYDxLDjHIJEB2AXKKmfEkOcUgkxhkFyCHmBlPklMs5BSDbGKQU0yM58gpVnKKQTaxkENMjOfIKVZyikE2sZBDTIznyCEu5BSDbGIlu5gYz5FDXMgpBtnESg5xMp4ju9jJIQbZxEoOcTKeI7vYySEG2cRKTnEwniO72MkuBjnESnZxMp4iuzjILhayiZWc4mA8Qw5xkF0MsouFnOJk/EwmcZBdDHIIkEmcjB/JJCayiUF2McgkTsZPZBYT2QXIJgaZxcn4gVyJiezilkxiYvxEJjGRQ3whu7hi/EgOcZJJ3GECccN4ikBckUk8z3iRzOJ5xqvkFL9gvEpO8QvGy2QTv2K8w4zfMv47478z/rt/WxorEMqsV+UAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAH4AAABJCAAAAADeN81BAAAB0UlEQVRoBcXBAWKiQADAwOT/j85pRVx60opQnDE+yfgk45OMTzIOYPKQ8SJjH5OnMn5lvE9+Ez8z3iSviR8Y75ANYpXxDtki1hibyWbxnLGVbBfPGZvIm+IZYwt5WzxhbCB7xH+M18ku8R/jZbJTfGe8SnaLb4xXyW7xjfESOUQsGa+Qo8TIeIEcJkbGb+RQMTC+McyMGzlWDIyRjALkaPFgjGQh5HDxYNzJSWJmfDE5STwYF3KmmBkg54o7A+RccWeAnCwmBsjJYmKAnCwmBsjZ4sYAOVvcGBdysrgxruRccWN8kVPFjXEjZ4obYyZnigvjQU4UF8aDnCnAWJDzBMZ3coKMC+MJ+TsxMFbIn4iRsU6OFyNjlfyFGBir5A/EyFglfyEGxioZJG8KWYiZsUpm8UU2iwtZiDtjlcxiIlvERBZiYqySWQxkRchF/EcWYmKsk7t4yjCM38lCfDFWySz2kqW4MtbJXewmS3FhrJO72E9GcWWsk7vYT0ZxZfxAJnEEGcSF8QOZxAFkFBfGKpnFAWQQV8YqmcV+MoorY4UMYjcZxI2xQgaxmwzixnhORrGf3MWd8ZwM4gAyiZmxRiZxELmKB+NExpLxScYnGZ/0D5gQsQFVm4eHAAAAAElFTkSuQmCC"" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAAA0CAAAAACcdVjuAAABT0lEQVRYCb3BB2KjAADAMPv/j/a1EFYz4QKScTnjcsbljC8yMzMzMx4w/o/JW7FmHGFIskNMjH0kOSBmxufkP8TE+JwcFzNjDzkqZsYOcljMjD3kqJgZO8hhMTM+J4fFwnhE4p4cFzPjESH+MDkuZsYjQpIkSfIjOSwWxgPybbEw7snXxcK4JyeJX8aWnCcGxoacKAbGhpwoBsbC5EwxMGZyshgYN3K6GBgjOV8MjJGcLwbGSM4XA2MiZ4uBcSOni4ExkksEGAgh1wgMkMsEGCCXCTBArhBCgIFcISbGQE4XN8ZIjkrei4Uxko/EA/JOrBgjeSfjMVkL5I9YMW7ktXhOVgLZijXjRl6L52QRIFuxZsxkkWzFSzKKgazFhjGTSSBb8YZATGQRW8aa/IofshJ7yST+MJ6RRewnv+KO8ZRM4puMF8yMLzMuZ1zOuNw/NtGOAeszs6kAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHQAAAA4CAAAAAAJa4PsAAABQElEQVRYCb3BAXLaMAAAwbv/P/pqYeOhCQkuqNo11jPWM9Yz1jPWM56SU8xmfGPyKCYzvpInYiLjb/JczGM8MPlJTGOcTH4Rsxh38ruYxTjJCzGHcZDXYgrjIK/FFMZOLokJjJ1cE58zdnJNfM7YyTXxOWOQy+Jjxo1cljyKf2bcyBxxMEAgvjEG+e/iZGxkidgZg6wRg7GRNeLGGGSR2BiDrBJgDLJKgDHIIrExNrJKbIxBFomNsZE14sbYyBKxMwaZIG7kB3EwBvlcHGQXIKe4Mwb5SELcCQkxyCFOxiBvibcYO/l38R5jJ9fEIJt4k7GTK+IgxLuMnVwQJ+NtxkFeijmMO3khJjHu5IWYxLiTX8U0xsnkRzGP8UCSZzLmMb6QJ2Im4yv5KuYyvjF5FJMZT8ghIWYz1jPWM9Yz1vsD0MZ8Aeb1ugwAAAAASUVORK5CYII="" width=""100"" >","<img src=""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHMAAABsCAAAAAB8qNJ0AAACX0lEQVRoBb3BAYLhQAAAwe7/P7pvHWGRxJCxVcafM/6c8eeMO2YY32TcyE18jXElv8W3GAu5F19iXMij+A7jTJ7FVxhnsiK+wTiTNfEFxomsiy8wQDbFfAbIppjPANkU8xnItpjPANkW0xn/yYaYzjiTdTGdcSbrYjrjQtbEfMaFrIrpjIWsiemMhayK2YyFrIvJjCvZEFMZV7IpJjJuZF9MYVzJa3GccSUj4ijjRsbEIcaVDIsDjCt5S3zIuJJ3xSeMhXwg3mcs5CPxLmMhH4r3GAv5VLzFuJAD4g3GQg6IccaFHBLDjIUcE4OMCzkoBhkLOSqGGBdyXIwwFjJBvGYsZIZ4yVjIFPGKsZA54gXjSuaIfcaVzBH7jCuZJHYZVzJL7DFuZJLYY/wik8QO4xeZJbYZv8ksscm4I7PEFuOeTBJbjHsyTawzHsg0scp4JLPEKuOJzBJrjCcyTawwnsk08cxYIdPEE2OVzBKPjFUyT9wz1sk8ccfYItPEb8YmmSdujG0yT1wZe2SaWBi7ZJq4MPbJNHFmvCKzxH/GazJJnBgjZIo4McbIuEzWBRijZEBcyKoAY5TsizuyJsAYJdvimawJjGFyEoNkRYAxTE5ilDwLMIbJjxgnzwJjlJzEOHkSYIySk3iDPAowRsmPeIs8CDAGyUm8Rx4ExiD5L94mNwHGIPkRn5Gz+GEMEojPCfGfMUYgpjDGCMQUxhD5EVMY+4TkImYw9sidmMHYJffiOGOLrIjjjA2yLo4yNsizMA4z1

Accuracies:


,elephant,turtle,classic,key,bird,brick,children,camel,fork,fountain,heart,car,bone,ray,hammer,face,misk,glas
Accuracy,,,,,,,,,,,,,,,,,,
0,0.4,0.6,1.0,1.0,0.2,0.4,1.0,0.2,0.6,1.0,0.6,0.8,1.0,1.0,0.8,1.0,0.6,1.0


Average along classes: 0.73


0.25